In [2]:
import pandas as pd
import numpy as np
import psycopg2
import pandas.io.sql as sqlio

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
#В целях безопасности адрес сервера, хост, логин и пароль не указаны

#conn = psycopg2.connect(.....)

In [ ]:
#Тестовые данные будут занимать 33%

In [1]:
# Выведем все заявки за определенны год
query = \
'''
SELECT 
Credit_requets.client_id,
Credit_requets.date,
Clients.sex,
date_part('year',age(Clients.date_birthday::date)) as age,
Credit_requets.flag_car as car,
Credit_requets.flag_car_is_import as car_type
SUM(Credit_requets.refusal) as decline_app_cnt
Credit_requets.income
Credit_requets.foreign_passport
Credit_requets.adress_city

FROM Credit_requets LEFT JOIN Clients ON
Credit_requets.client_id = Clients.client_id

HAVING date_part('year', date) = 2014
'''


In [ ]:
#Создадим запрос с информацией из БКИ, путем вложенного подзапроса, т.к. такую информацию как id наших клиентов БКИ
#не хранит
query = \
'''
SELECT Clients.client_id,
Bki.score_bki,
bki_request_cnt
FROM Clients INNER JOIN Bki ON Clients.inn = Bki.inn
'''

In [ ]:
#Соединим запросы
query = \
'''
SELECT query_requests.client_id, 
query_requests.date, 
query_requests.sex, 
query_requests.age, 
query_requests.car, 
query_requests.car_type, 
query_requests.decline_app_cnt,
query_requests.income,
query_requests.foreign_passport,
query_requests.adress_sity
FROM
    (
    SELECT 
    Credit_requets.client_id,
    Credit_requets.date,
    Clients.sex,
    date_part('year',age(Clients.date_birthday::date)) as age,
    Credit_requets.flag_car as car,
    Credit_requets.flag_car_is_import as car_type,
    SUM(Credit_requets.refusal) as decline_app_cnt,
    Credit_requets.income,
    Credit_requets.foreign_passport

    FROM Credit_requets LEFT JOIN Clients ON
    Credit_requets.client_id = Clients.client_id

    HAVING date_part('year', date) = 2014
    ) as query_requests
LEFT JOIN 
    (
    SELECT Clients.client_id,
    Bki.score_bki,
    bki_request_cnt
    FROM Clients INNER JOIN Bki ON Clients.inn = Bki.inn
    ) as query_bki
ON query_requests.client_id = query_bki.client_id
'''

In [ ]:
#Создадим запрос с информацией о рейтинге городов и заджойним с предыдущим
query = \
'''
SELECT Clients.client_id,
Clients.adress_city,
City.city_rating as rating
FROM Clients LEFT JOIN City ON Clients.id_city = City.id_city
'''

In [ ]:
#Соединим запросы
query = \
'''
SELECT main_query.client_id,
main_query.date, 
main_query.sex, 
main_query.age, 
main_query.car, 
main_query.car_type, 
main_query.decline_app_cnt,
main_query.score_bki,
main_query.bki_request_cnt,
query_city.rating,
main_query.income,
main_query.foreign_passport,
query_requests.adress_city,
main_query.default
FROM
    (
    SELECT query_requests.client_id, 
    query_requests.date, 
    query_requests.sex, 
    query_requests.age, 
    query_requests.car, 
    query_requests.car_type, 
    query_requests.decline_app_cnt,
    Bki.score_bki,
    Bki.bki_request_cnt,
    query_requests.income,
    query_requests.foreign_passport,
    query_requests.adress_city,
    query_requests.done as default
    FROM
        (
        SELECT 
        Credit_requets.client_id,
        Credit_requets.date,
        Clients.sex,
        date_part('year',age(Clients.date_birthday::date)) as age,
        Credit_requets.flag_car as car,
        Credit_requets.flag_car_is_import as car_type,
        SUM(Credit_requets.refusal) as decline_app_cnt,
        Credit_requets.income,
        Credit_requets.foreign_passport,
        Credit_requets.done

        FROM Credit_requets LEFT JOIN Clients ON
        Credit_requets.client_id = Clients.client_id

        HAVING date_part('year', date) = 2014
        ) as query_requests
    LEFT JOIN 
        (
        SELECT Clients.client_id,
        Bki.score_bki,
        Bki.bki_request_cnt
        FROM Clients INNER JOIN Bki ON Clients.inn = Bki.inn
        ) as query_bki
    ON query_requests.client_id = query_bki.client_id
    ) as main_query
LEFT JOIN
    (
    SELECT Clients.client_id,
    Clients.adress_city,
    City.city_rating as rating
    FROM Clients LEFT JOIN City ON Clients.id_city = City.id_city
    ) as query_city
'''

In [ ]:
df = sqlio.read_sql_qeury(query, conn)
'''
Данные о выплате кредита были взяты из хранилища данных с заявками на кредит. На каждой заявке есть флаг done. 
Он означает закрытие кредита по этой заявке, нам надо выявить наоборот не выплаченные, поэтому поменяем местами 0 и 1.
'''
df = df['default'].replace({0:1, 1:0})

In [83]:
'''
client_id - id клиента
date - дата заявки на кредит
sex - пол (M - муж, F - жен)
age - возраст
car - флаг автомобиля (Y - есть автомобиль, N - нет автомобиля)
car_type - флаг иномарки (Y - иномарка, N - не иномарка или нет автомобиля)
decline_app_cnt - количество отказанных прошлых заявок
score_bki - скоринговый балл по данным из БКИ
bki_request_cnt - количество запросов в БКИ
region_rating - рейтинг региона
income - доход заемщика
foreign_passport - наличие загранпаспорта
default - флаг дефолта по кредиту
'''

'\nclient_id - id клиента\ndate - дата заявки на кредит\nsex - пол (M - муж, F - жен)\nage - возраст\ncar - флаг автомобиля (Y - есть автомобиль, N - нет автомобиля)\ncar_type - флаг иномарки (Y - иномарка, N - не иномарка или нет автомобиля)\ndecline_app_cnt - количество отказанных прошлых заявок\nscore_bki - скоринговый балл по данным из БКИ \nbki_request_cnt - количество запросов в БКИ\nregion_rating - рейтинг региона\nincome - доход заемщика\nforeign_passport - наличие загранпаспорта\ndefault - флаг дефолта по кредиту\n'

In [25]:
'''
Без знания предметной области сложно сходу узнать корреляцию скорингового балла и кредитоспособность клиента.
Поэтому воспользуемся скриптом в ознакомительных для себя целях.
'''

if df.query('default == 0')['score_bki'].mean() < df.query('default == 1')['score_bki'].mean():
    print('Чем ниже скоринговый балл по данным из БКИ, тем клиент безопаснее')
else:
    print('Чем выше скоринговый балл по данным из БКИ, тем клиент безопаснее')

if df.query('default == 0')['region_rating'].mean() < df.query('default == 1')['region_rating'].mean():
    print('Чем ниже рейтинг региона, тем клиент безопаснее')
else:
    print('Чем выше рейтинг региона, тем клиент безопаснее')
    


Чем ниже скоринговый балл по данным из БКИ, тем клиент безопаснее
Чем выше рейтинг региона, тем клиент безопаснее


In [26]:
# Заменим значения столбцов на их булевые аналоги
df.loc[(df_train.sex == 'M'), 'sex'] = 1
df.loc[(df_train.sex == 'F'), 'sex'] = 0
df.loc[(df_train.car == 'Y'), 'car'] = 1
df.loc[(df_train.car == 'N'), 'car'] = 0
df.loc[(df_train.car_type == 'Y'), 'car_type'] = 1
df.loc[(df_train.car_type == 'N'), 'car_type'] = 0
df.loc[(df_train.foreign_passport == 'Y'), 'foreign_passport'] = 1
df.loc[(df_train.foreign_passport == 'N'), 'foreign_passport'] = 0

In [98]:
'''
Чтобы посмотреть данные, сохранил их в файле. Файл выложил в свой Github
ссылка https://github.com/Iladullin/diplom2023
'''
df.to_csv('df.csv', index=False)

In [15]:
df['day'] = df['date'].str[:2].astype('int')
df['mounth'] = df['date'].str[3:5].astype('int')
df = df.drop(columns=['date'])

In [16]:
df = df[['client_id', 'day', 'mounth', 'sex', 'age', 'car', 'car_type', 'decline_app_cnt', 'score_bki', 'bki_request_cnt'\
    , 'region_rating', 'income', 'foreign_passport', 'default']]

In [17]:
df

,client_id,day,mounth,sex,age,car,car_type,decline_app_cnt,score_bki,bki_request_cnt,region_rating,income,foreign_passport,default
0,25905,1,2,1,62,1,1,0,-2.008753,1,50,18000,0,0
1,63161,12,3,0,59,0,0,0,-1.532276,3,50,19000,0,0
2,25887,1,2,1,25,1,0,2,-1.408142,1,80,30000,1,0
3,16222,23,1,0,53,0,0,0,-2.057471,2,50,10000,0,0
4,101655,18,4,1,48,0,0,0,-1.244723,1,60,30000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73794,54887,4,3,0,45,0,0,0,-1.792064,3,50,17000,0,0
73795,76821,24,3,1,41,1,1,0,-2.058029,1,50,70000,0,0
73796,103695,22,4,1,31,0,0,0,-1.512635,4,80,45000,0,0
73797,861,4,1,0,29,0,0,0,-1.479334,3,50,13000,0,1


In [18]:
# Проведем предварительный анализ датафрейма
df.describe()

,client_id,day,mounth,sex,age,car,car_type,decline_app_cnt,score_bki,bki_request_cnt,region_rating,income,foreign_passport,default
count,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000,73799.000000
mean,55137.965094,15.966517,2.537053,0.436822,39.280641,0.324760,0.189813,0.275749,-1.904724,2.000339,56.725701,41099.775417,0.149948,0.126994
std,31841.921168,8.602978,1.089098,0.495996,11.520378,0.468288,0.392156,0.804272,0.498231,2.252073,13.055717,46166.322400,0.357023,0.332968
min,1.000000,1.000000,1.000000,0.000000,21.000000,0.000000,0.000000,0.000000,-3.624586,0.000000,20.000000,1000.000000,0.000000,0.000000
25%,27440.500000,9.000000,2.000000,0.000000,30.000000,0.000000,0.000000,0.000000,-2.259534,0.000000,50.000000,20000.000000,0.000000,0.000000
50%,55274.000000,16.000000,3.000000,0.000000,37.000000,0.000000,0.000000,0.000000,-1.920823,1.000000,50.000000,30000.000000,0.000000,0.000000
75%,82758.500000,23.000000,3.000000,1.000000,48.000000,1.000000,0.000000,0.000000,-1.569831,3.000000,60.000000,48000.000000,0.000000,0.000000
max,110147.000000,31.000000,4.000000,1.000000,72.000000,1.000000,1.000000,33.000000,0.199773,53.000000,80.000000,1000000.000000,1.000000,1.000000


describe() проводит анализ датафрейма, точнее только его численных параметров

Выводит таблицу из столбцов, которые относятся к числовому типу и проводит анализ, выводя таблицу со строками
count - кол-во элементоа
mean - среднее значение
std - стандартное отклонение

Чтобы знать что такое стандартное отклонение нужно знать
Дисперсия - средний квадрат отклонений индивидуальных значений
Формула: sum((i - avg)**2)/count-1

Стандартное отклонение - квардратный корень из дисперсии
Как считается дисперсия:
Допустим есть множество из 7 чисел (1 2 2 3 4 4 5)

Среднее значение (avg)= (1 + 2 + 2 + 3 + 4 + 4 + 5)/7  = 21/7 = 3

(1 - 3)**2 = 4

(2 - 3)**2 = 1

(2 - 3)**2 = 1

(3 - 3)**2 = 0

(4 - 3)**2 = 1

(4 - 3)**2 = 1

(5 - 3)**2 = 4

Дисперсия = (4 + 1 + 1 + 0 + 1 + 1 + 4)/7-1 = 12/6 = 2
Стандартное отклонение - корень из 2 = 1.4

min - минимальное значение
max - максимальное значение

Также в методе расчитаны квартили (25, 50 и 75 процентов)
Квартили показывают:
25% - 25  процентов всех значений имеют показатель меньше данного числа, а 75 - больше
25% - 50  процентов всех значений имеют показатель меньше данного числа, а 50 - больше
25% - 75  процентов всех значений имеют показатель меньше данного числа, а 25 - больше

In [19]:
# В результате датафрейм приобретает вид полностью численный. Его можно посчитать

In [20]:
# Разделение датафрейма на X и y серию

X = df.drop('default', axis=1)
y = df['default']
y = y.astype('int')

In [21]:
# Деление датафрейма на тренировочный и тестовый

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)


In [22]:
X_train

,client_id,day,mounth,sex,age,car,car_type,decline_app_cnt,score_bki,bki_request_cnt,region_rating,income,foreign_passport
8931,104346,22,4,0,27,0,0,0,-2.456906,3,60,30000,0
63915,56248,5,3,0,47,0,0,0,-1.988478,4,50,20000,0
45568,59433,7,3,0,47,0,0,0,-2.225004,0,80,45000,0
3391,21096,28,1,0,32,0,0,0,-2.572445,3,50,21000,0
40660,18878,26,1,0,31,0,0,0,-1.694271,6,80,30000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,15225,22,1,1,28,0,0,1,-1.937027,5,40,32000,0
6265,1002,4,1,0,54,1,0,0,-2.095654,6,50,80000,0
54886,16226,23,1,1,30,0,0,0,-2.199236,4,80,45000,0
860,3695,9,1,1,35,1,0,0,-2.299785,1,50,30000,0


In [23]:
y_train

8931     0
63915    0
45568    0
3391     1
40660    0
        ..
37194    0
6265     0
54886    0
860      0
15795    0
Name: default, Length: 49445, dtype: int32

In [24]:
X_test

,client_id,day,mounth,sex,age,car,car_type,decline_app_cnt,score_bki,bki_request_cnt,region_rating,income,foreign_passport
68068,66799,15,3,0,29,0,0,1,-1.195981,3,40,23000,0
60795,25379,1,2,1,58,0,0,0,-1.316277,2,50,15000,0
5365,28825,4,2,0,69,0,0,0,-1.812376,0,80,23500,0
27565,37530,13,2,1,27,1,1,0,-1.077649,0,50,65000,0
72235,4264,10,1,0,32,0,0,1,-1.516908,6,50,25000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50028,96657,13,4,1,48,1,1,0,-1.670952,0,70,25000,0
56072,65908,14,3,1,49,0,0,0,-2.345798,3,50,75000,1
28691,74542,22,3,1,28,0,0,0,-2.374462,1,80,85000,0
13045,82350,30,3,1,45,1,1,1,-1.747502,0,80,50000,1


In [25]:
y_test

68068    0
60795    0
5365     1
27565    0
72235    1
        ..
50028    1
56072    0
28691    0
13045    0
43196    0
Name: default, Length: 24354, dtype: int32

In [26]:
# Создание классификатора

rf_clf = RandomForestClassifier(random_state=0, criterion='entropy')

In [27]:
# Создание параметров для классификатора

parametrs = {'n_estimators': range(10,25,3), 
         'max_depth': range(1,9,4), 
         'min_samples_leaf': range(1,2,1), 
         'min_samples_split': range(2,3,1)} 

In [28]:
# Создание классификатора оптимальной длины

grid_search_cv_clf = GridSearchCV(rf_clf, parametrs, n_jobs=-1, cv=3)

In [29]:
# Обучение классификатора

grid_search_cv_clf.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(criterion='entropy',
                                              random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': range(1, 9, 4),
                         'min_samples_leaf': range(1, 2),
                         'min_samples_split': range(2, 3),
                         'n_estimators': range(10, 25, 3)})

In [30]:
# Предсказывание данных на тестовом датафрейме и помещения в него столбца с полученным значением

predictions = grid_search_cv_clf.predict(X_test)
X_test['pred'] = predictions

<ipython-input-30-46286b3bccd8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['pred'] = predictions


In [31]:
# Создание датафрейма с полученными данными для получения выборки и сводной

result = y_test.to_frame()
result['pred'] = X_test['pred']

In [33]:
# Создание столбца со значением совпадения ожидаемого и получившегося результата

result['yes/no'] = result['default'] == result['pred']
result['yes/no'] = result['yes/no'].astype(int)

In [34]:
# Получение значения точности в процентах.

result['yes/no'].sum()/result['yes/no'].count() * 100

87.43532889874353

 При текущей задаче точность получилось около 87% (87.43532889874353)

In [35]:
result

,default,pred,yes/no
68068,0,0,1
60795,0,0,1
5365,1,0,0
27565,0,0,1
72235,1,0,0
...,...,...,...
50028,1,0,0
56072,0,0,1
28691,0,0,1
13045,0,0,1


In [2]:
# Данные о результатах (в файле создан лист со сводной диаграммой). Файл также загружен на Github
result.to_excel('result.xlsx')

In [ ]:
# Также блокнот с данным кодом будет дополнительно загружен на Github
# ссылка - https://github.com/Iladullin/diplom2023